### AutoGen vs Agent Framework: Agent-as-a-Tool pattern📌

##### Agent-as-Tool Pattern Explained
This code demonstrates hierarchical agent architectures - a powerful pattern where a "coordinator" agent delegates specialized tasks to "sub-agents" by treating them as tools. Think of it as a manager (coordinator) who assigns work to specialists (sub-agents).

##### Core Concept: Agent-as-Tool
Instead of giving agents simple functions like `search_database()` or `send_email()`, you give them entire AI agents as tools. The coordinator agent can then delegate complex reasoning tasks to specialized agents.

Why is this powerful?

* Specialization: Each agent focuses on one domain (writing, analysis, coding)
* Modularity: Easily add/remove specialist agents
* Scalability: Complex tasks broken into manageable sub-tasks
* Reusability: Same specialist agents used by multiple coordinators

In [ ]:
# Cell 1: Install required packages
! pip install autogen-agentchat autogen-ext[openai] agent-framework openai python-dotenv beautifulsoup4 -q
print("✅ All packages installed successfully!")

In [1]:
# Cell 2: Import all required libraries
import asyncio
import os
import json
import re
from typing import Dict, List, Optional
from datetime import datetime
from dotenv import load_dotenv

print("✅ All libraries imported successfully!")

✅ All libraries imported successfully!


In [2]:
load_dotenv()

True

In [3]:
# Configure API Key
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise ValueError("Please set the OPENAI_API_KEY environment variable.")

print("✅ API Key configured!")

✅ API Key configured!


In [4]:
# Define tools for each specialist agent

import json
from datetime import datetime

# Research Database (simulated)
RESEARCH_DB = {
    "artificial intelligence": {
        "stats": {
            "market_size_2024": "$196.63 billion",
            "growth_rate": "37.3% CAGR",
            "adoption_rate": "35% of companies"
        },
        "trends": [
            "Generative AI explosion",
            "AI regulation debates",
            "Enterprise AI adoption",
            "AI ethics concerns"
        ],
        "sources": [
            {"title": "AI Market Report 2024", "url": "research.com/ai-report"},
            {"title": "Enterprise AI Trends", "url": "techreview.com/ai-trends"}
        ]
    },
    "remote work": {
        "stats": {
            "remote_workers_2024": "35% of US workforce",
            "productivity_increase": "13%",
            "cost_savings": "$11,000 per employee"
        },
        "trends": [
            "Hybrid work models",
            "Digital nomad lifestyle",
            "Work-life balance focus",
            "Collaboration tools evolution"
        ],
        "sources": [
            {"title": "Remote Work Statistics", "url": "workstats.com/remote"},
            {"title": "Hybrid Work Guide", "url": "businessinsider.com/hybrid"}
        ]
    },
    "sustainability": {
        "stats": {
            "renewable_energy_2024": "30% of global energy",
            "ev_adoption": "18% of new car sales",
            "corporate_net_zero": "65% committed"
        },
        "trends": [
            "Carbon neutrality goals",
            "Circular economy models",
            "Green technology investment",
            "ESG reporting standards"
        ],
        "sources": [
            {"title": "Sustainability Report 2024", "url": "greentech.com/report"},
            {"title": "Net Zero Strategies", "url": "climate.org/netzero"}
        ]
    }
}

# Content Database
CONTENT_DB = {}

# SEO Database
SEO_DB = {}


# ============================================================================
# RESEARCHER TOOLS
# ============================================================================

def research_topic(topic: str, depth: str = "standard") -> str:
    """
    Research a topic and gather data, statistics, and sources.
    
    Args:
        topic: Topic to research
        depth: Research depth (basic, standard, comprehensive)
    
    Returns:
        JSON string with research findings
    """
    print(f"🔬 Researching topic: {topic} (depth: {depth})")
    
    # Find matching research
    research_data = None
    for key, data in RESEARCH_DB.items():
        if key.lower() in topic.lower() or topic.lower() in key.lower():
            research_data = data
            break
    
    if not research_data:
        research_data = {
            "stats": {"general_info": "Limited data available"},
            "trends": ["Emerging topic", "Growing interest"],
            "sources": [{"title": "General Overview", "url": "example.com"}]
        }
    
    findings = {
        "topic": topic,
        "depth": depth,
        "statistics": research_data["stats"],
        "current_trends": research_data["trends"],
        "sources": research_data["sources"],
        "researched_at": datetime.now().isoformat()
    }
    
    return json.dumps(findings, indent=2)


def fact_check(statement: str) -> str:
    """
    Verify the accuracy of a statement.
    
    Args:
        statement: Statement to fact-check
    
    Returns:
        JSON string with verification result
    """
    print(f"✓ Fact-checking: {statement[:60]}...")
    
    # Simulated fact-checking
    confidence = 0.85
    
    return json.dumps({
        "statement": statement[:150],
        "verified": True,
        "confidence": confidence,
        "status": "accurate" if confidence > 0.7 else "needs_review",
        "checked_at": datetime.now().isoformat()
    }, indent=2)


def find_citations(topic: str, count: int = 3) -> str:
    """
    Find credible citations for a topic.
    
    Args:
        topic: Topic to find citations for
        count: Number of citations needed
    
    Returns:
        JSON string with citations
    """
    print(f"📚 Finding citations for: {topic}")
    
    # Find relevant sources
    citations = []
    for key, data in RESEARCH_DB.items():
        if key.lower() in topic.lower():
            citations = data["sources"][:count]
            break
    
    if not citations:
        citations = [
            {"title": f"{topic} Overview", "url": "academic.edu/overview"},
            {"title": f"{topic} Research", "url": "journal.com/research"}
        ]
    
    return json.dumps({
        "topic": topic,
        "citations_found": len(citations),
        "citations": citations
    }, indent=2)


# ============================================================================
# WRITER TOOLS
# ============================================================================

def generate_outline(topic: str, target_length: int = 1000) -> str:
    """
    Generate a structured outline for an article.
    
    Args:
        topic: Article topic
        target_length: Target word count
    
    Returns:
        JSON string with outline
    """
    print(f"📝 Generating outline for: {topic}")
    
    outline = {
        "title": f"The Complete Guide to {topic.title()}",
        "meta_description": f"Discover everything you need to know about {topic} in this comprehensive guide.",
        "target_words": target_length,
        "structure": [
            {
                "section": "Introduction",
                "estimated_words": int(target_length * 0.15),
                "key_points": ["Hook", "Problem statement", "Article overview"]
            },
            {
                "section": "Main Content",
                "estimated_words": int(target_length * 0.60),
                "subsections": [
                    "What is " + topic.title(),
                    "Key Benefits and Advantages",
                    "Current Trends and Statistics",
                    "Best Practices"
                ]
            },
            {
                "section": "Conclusion",
                "estimated_words": int(target_length * 0.15),
                "key_points": ["Summary", "Call to action", "Future outlook"]
            }
        ],
        "generated_at": datetime.now().isoformat()
    }
    
    return json.dumps(outline, indent=2)


def save_draft(title: str, content: str, author: str = "Content Team") -> str:
    """
    Save article draft.
    
    Args:
        title: Article title
        content: Article content
        author: Author name
    
    Returns:
        JSON string with save confirmation
    """
    print(f"💾 Saving draft: {title}")
    
    draft_id = f"DRAFT{datetime.now().strftime('%Y%m%d%H%M%S')}"
    
    CONTENT_DB[draft_id] = {
        "draft_id": draft_id,
        "title": title,
        "content": content,
        "author": author,
        "word_count": len(content.split()),
        "status": "draft",
        "created_at": datetime.now().isoformat()
    }
    
    return json.dumps({
        "status": "saved",
        "draft_id": draft_id,
        "title": title,
        "word_count": len(content.split()),
        "message": "Draft saved successfully"
    }, indent=2)


# ============================================================================
# EDITOR TOOLS
# ============================================================================

def check_grammar(text: str) -> str:
    """
    Check grammar and style issues.
    
    Args:
        text: Text to check
    
    Returns:
        JSON string with grammar report
    """
    print(f"✏️  Checking grammar ({len(text)} characters)...")
    
    # Simulated grammar check
    issues = []
    
    # Simple checks
    if text.count("very") > 2:
        issues.append({
            "type": "word_choice",
            "issue": "Overuse of 'very'",
            "suggestion": "Use stronger adjectives instead"
        })
    
    if len([s for s in text.split('.') if len(s.split()) > 30]) > 0:
        issues.append({
            "type": "readability",
            "issue": "Long sentences detected",
            "suggestion": "Break into shorter sentences"
        })
    
    return json.dumps({
        "text_length": len(text),
        "issues_found": len(issues),
        "issues": issues,
        "overall_score": 85 if len(issues) < 3 else 70,
        "status": "good" if len(issues) < 3 else "needs_improvement"
    }, indent=2)


def check_readability(text: str) -> str:
    """
    Analyze text readability.
    
    Args:
        text: Text to analyze
    
    Returns:
        JSON string with readability metrics
    """
    print(f"📊 Analyzing readability...")
    
    words = text.split()
    sentences = text.split('.')
    
    avg_words_per_sentence = len(words) / max(len(sentences), 1)
    
    # Simulated readability metrics
    metrics = {
        "word_count": len(words),
        "sentence_count": len(sentences),
        "avg_words_per_sentence": round(avg_words_per_sentence, 1),
        "estimated_reading_time": f"{len(words) // 200} min",
        "readability_score": "Good" if avg_words_per_sentence < 20 else "Needs improvement",
        "grade_level": "8-10th grade"
    }
    
    return json.dumps(metrics, indent=2)


def suggest_improvements(content: str) -> str:
    """
    Suggest content improvements.
    
    Args:
        content: Content to review
    
    Returns:
        JSON string with suggestions
    """
    print(f"💡 Generating improvement suggestions...")
    
    suggestions = [
        {
            "category": "structure",
            "suggestion": "Add more subheadings for better scannability"
        },
        {
            "category": "engagement",
            "suggestion": "Include a compelling call-to-action in conclusion"
        },
        {
            "category": "clarity",
            "suggestion": "Define technical terms for broader audience"
        }
    ]
    
    return json.dumps({
        "suggestions_count": len(suggestions),
        "suggestions": suggestions,
        "priority": "medium"
    }, indent=2)


# ============================================================================
# SEO SPECIALIST TOOLS
# ============================================================================

def analyze_seo(content: str, target_keyword: str) -> str:
    """
    Analyze content for SEO optimization.
    
    Args:
        content: Content to analyze
        target_keyword: Primary keyword
    
    Returns:
        JSON string with SEO analysis
    """
    print(f"🔍 Analyzing SEO for keyword: {target_keyword}")
    
    keyword_count = content.lower().count(target_keyword.lower())
    word_count = len(content.split())
    keyword_density = (keyword_count / word_count * 100) if word_count > 0 else 0
    
    analysis = {
        "target_keyword": target_keyword,
        "keyword_count": keyword_count,
        "keyword_density": f"{keyword_density:.2f}%",
        "optimal_density": "1-2%",
        "status": "good" if 1 <= keyword_density <= 2 else "needs_optimization",
        "recommendations": []
    }
    
    if keyword_density < 1:
        analysis["recommendations"].append("Increase keyword usage")
    elif keyword_density > 2:
        analysis["recommendations"].append("Reduce keyword stuffing")
    
    analysis["recommendations"].append("Add keyword to first paragraph")
    analysis["recommendations"].append("Include keyword in at least one subheading")
    
    return json.dumps(analysis, indent=2)


def generate_meta_tags(title: str, content: str) -> str:
    """
    Generate SEO meta tags.
    
    Args:
        title: Article title
        content: Article content
    
    Returns:
        JSON string with meta tags
    """
    print(f"🏷️  Generating meta tags for: {title}")
    
    # Extract first sentence for description
    first_sentence = content.split('.')[0][:160] if content else title
    
    meta_tags = {
        "meta_title": f"{title} | Expert Guide",
        "meta_description": first_sentence,
        "og_title": title,
        "og_description": first_sentence,
        "twitter_card": "summary_large_image",
        "canonical_url": f"https://blog.example.com/{title.lower().replace(' ', '-')}"
    }
    
    return json.dumps(meta_tags, indent=2)


def suggest_keywords(topic: str, count: int = 5) -> str:
    """
    Suggest related keywords.
    
    Args:
        topic: Main topic
        count: Number of keywords to suggest
    
    Returns:
        JSON string with keyword suggestions
    """
    print(f"🔑 Suggesting keywords for: {topic}")
    
    # Simulated keyword research
    keywords = [
        {"keyword": f"{topic} guide", "search_volume": "5,400", "difficulty": "medium"},
        {"keyword": f"best {topic}", "search_volume": "3,200", "difficulty": "high"},
        {"keyword": f"{topic} tips", "search_volume": "2,800", "difficulty": "low"},
        {"keyword": f"how to {topic}", "search_volume": "4,100", "difficulty": "medium"},
        {"keyword": f"{topic} benefits", "search_volume": "1,900", "difficulty": "low"}
    ]
    
    return json.dumps({
        "primary_topic": topic,
        "keywords_found": len(keywords),
        "keywords": keywords[:count]
    }, indent=2)


print("✅ All specialist tools defined!")
print("\n📋 Tool Categories:")
print("   🔬 Researcher: research_topic, fact_check, find_citations")
print("   📝 Writer: generate_outline, save_draft")
print("   ✏️  Editor: check_grammar, check_readability, suggest_improvements")
print("   🔍 SEO: analyze_seo, generate_meta_tags, suggest_keywords")


✅ All specialist tools defined!

📋 Tool Categories:
   🔬 Researcher: research_topic, fact_check, find_citations
   📝 Writer: generate_outline, save_draft
   ✏️  Editor: check_grammar, check_readability, suggest_improvements
   🔍 SEO: analyze_seo, generate_meta_tags, suggest_keywords


In [5]:
# Agent Framework - Content Marketing Agency

async def run_agent_framework_content_agency():
    """
    Agent Framework implementation of hierarchical content creation.
    Manager delegates to Researcher → Writer → Editor → SEO Specialist.
    """
    from agent_framework import ai_function
    from agent_framework.openai import OpenAIChatClient
    
    print("\n" + "="*70)
    print("⚡ AGENT FRAMEWORK - CONTENT MARKETING AGENCY")
    print("="*70)
    print("\nHierarchical Structure:")
    print("  Manager Agent")
    print("    ├── Researcher Agent")
    print("    ├── Writer Agent")
    print("    ├── Editor Agent")
    print("    └── SEO Specialist Agent")
    print("\n" + "-"*70)
    
    # Decorate all tools
    @ai_function
    def research_topic_ai(topic: str, depth: str = "standard") -> str:
        """Research a topic."""
        return research_topic(topic, depth)
    
    @ai_function
    def fact_check_ai(statement: str) -> str:
        """Verify facts."""
        return fact_check(statement)
    
    @ai_function
    def find_citations_ai(topic: str, count: int = 3) -> str:
        """Find citations."""
        return find_citations(topic, count)
    
    @ai_function
    def generate_outline_ai(topic: str, target_length: int = 1000) -> str:
        """Generate outline."""
        return generate_outline(topic, target_length)
    
    @ai_function
    def save_draft_ai(title: str, content: str, author: str = "Content Team") -> str:
        """Save draft."""
        return save_draft(title, content, author)
    
    @ai_function
    def check_grammar_ai(text: str) -> str:
        """Check grammar."""
        return check_grammar(text)
    
    @ai_function
    def check_readability_ai(text: str) -> str:
        """Check readability."""
        return check_readability(text)
    
    @ai_function
    def suggest_improvements_ai(content: str) -> str:
        """Suggest improvements."""
        return suggest_improvements(content)
    
    @ai_function
    def analyze_seo_ai(content: str, target_keyword: str) -> str:
        """Analyze SEO."""
        return analyze_seo(content, target_keyword)
    
    @ai_function
    def generate_meta_tags_ai(title: str, content: str) -> str:
        """Generate meta tags."""
        return generate_meta_tags(title, content)
    
    @ai_function
    def suggest_keywords_ai(topic: str, count: int = 5) -> str:
        """Suggest keywords."""
        return suggest_keywords(topic, count)
    
    # Create client
    client = OpenAIChatClient(model_id="gpt-4o-mini")
    
    # Create specialized agents
    researcher = client.create_agent(
        name="researcher",
        instructions="""You are a research specialist.

Your role:
1. Research topics thoroughly
2. Gather statistics and data
3. Find credible sources and citations
4. Fact-check information

Be thorough and cite your sources.""",
        tools=[research_topic_ai, fact_check_ai, find_citations_ai],
        model="gpt-4o-mini"
    )
    
    writer = client.create_agent(
        name="writer",
        instructions="""You are a professional content writer.

Your role:
1. Create structured outlines
2. Write engaging, clear content
3. Follow SEO best practices
4. Save drafts

Write in an engaging, professional style.""",
        tools=[generate_outline_ai, save_draft_ai],
        model="gpt-4.1"
    )
    
    editor = client.create_agent(
        name="editor",
        instructions="""You are a content editor.

Your role:
1. Check grammar and style
2. Analyze readability
3. Suggest improvements
4. Ensure quality standards

Be thorough but constructive.""",
        tools=[check_grammar_ai, check_readability_ai, suggest_improvements_ai],
        model="gpt-4.1"
    )
    
    seo_specialist = client.create_agent(
        name="seo_specialist",
        instructions="""You are an SEO specialist.

Your role:
1. Analyze content for SEO
2. Generate meta tags
3. Suggest keywords
4. Optimize for search engines

Focus on modern SEO best practices.""",
        tools=[analyze_seo_ai, generate_meta_tags_ai, suggest_keywords_ai],
        model="gpt-4.1"
    )
    
    # Convert specialists to tools
    researcher_tool = researcher.as_tool(
        name="research_specialist",
        description="Expert researcher who gathers data, statistics, and sources",
        arg_name="research_request",
        arg_description="What to research"
    )
    
    writer_tool = writer.as_tool(
        name="content_writer",
        description="Professional writer who creates outlines and content",
        arg_name="writing_request",
        arg_description="What to write"
    )
    
    editor_tool = editor.as_tool(
        name="content_editor",
        description="Editor who reviews and improves content quality",
        arg_name="editing_request",
        arg_description="What to edit or review"
    )
    
    seo_tool = seo_specialist.as_tool(
        name="seo_optimizer",
        description="SEO expert who optimizes content for search engines",
        arg_name="seo_request",
        arg_description="What to optimize"
    )
    
    # Create manager agent with all specialists as tools
    manager = client.create_agent(
        name="content_manager",
        instructions="""You are the Content Marketing Manager coordinating a team of specialists.

Your team:
- Research Specialist: Gathers data and sources
- Content Writer: Creates outlines and writes content
- Content Editor: Reviews and improves quality
- SEO Optimizer: Optimizes for search engines

Your workflow:
1. Delegate research to the researcher
2. Have the writer create an outline and content based on research
3. Have the editor review the content
4. Have the SEO specialist optimize it
5. Summarize the final output

Coordinate the team efficiently and present the final result.""",
        tools=[researcher_tool, writer_tool, editor_tool, seo_tool],
        model="gpt-4.1"
    )
    
    # Execute content creation workflow
    task = "Create a blog post about 'artificial intelligence in business' - 800 words"
    
    print(f"\n📋 Task: {task}\n")
    print("🚀 Manager delegating to specialists...\n")
    print("="*70)
    
    # Stream the entire workflow
    async for chunk in manager.run_stream(task):
        if chunk.text:
            print(chunk.text, end="", flush=True)
    
    print("\n\n" + "="*70)
    print("✅ CONTENT CREATION COMPLETED")
    print("="*70)
    print(f"\n📊 Drafts created: {len(CONTENT_DB)}")
    if CONTENT_DB:
        for draft_id, draft in CONTENT_DB.items():
            print(f"\n  {draft_id}:")
            print(f"    Title: {draft['title']}")
            print(f"    Words: {draft['word_count']}")
            print(f"    Status: {draft['status']}")
    
    print("\n💡 What happened:")
    print("   1. Manager delegated research to Researcher")
    print("   2. Researcher gathered data and sources")
    print("   3. Writer created outline and content")
    print("   4. Editor reviewed quality")
    print("   5. SEO Specialist optimized content")
    print("   6. Manager coordinated everything!")


print("✅ Agent Framework content agency defined!")
print("🚀 Run with: await run_agent_framework_content_agency()")


✅ Agent Framework content agency defined!
🚀 Run with: await run_agent_framework_content_agency()


In [12]:
# AutoGen - Content Marketing Agency

async def run_autogen_content_agency():
    """
    AutoGen implementation with AgentTool pattern.
    """
    from autogen_agentchat.agents import AssistantAgent
    from autogen_agentchat.tools import AgentTool  # AgentTool from agentchat.tools
    from autogen_core.tools import FunctionTool      # FunctionTool from core.tools
    from autogen_ext.models.openai import OpenAIChatCompletionClient
    
    print("\n" + "="*70)
    print("🤖 AUTOGEN - CONTENT MARKETING AGENCY")
    print("="*70)
    print("\nHierarchical Structure:")
    print("  Manager Agent")
    print("    ├── Researcher Agent")
    print("    ├── Writer Agent")
    print("    ├── Editor Agent")
    print("    └── SEO Agent")
    print("\n" + "-"*70)
    
    # Create model client
    client = OpenAIChatCompletionClient(model="gpt-4o-mini")
    
    # Create Researcher agent
    researcher = AssistantAgent(
        name="researcher",
        model_client=client,
        tools=[
            FunctionTool(func=research_topic, description="Research topics"),
            FunctionTool(func=fact_check, description="Verify facts"),
            FunctionTool(func=find_citations, description="Find citations")
        ],
        system_message="""You are a research specialist.

Gather data, statistics, and credible sources for topics.""",
        model_client_stream=True
    )
    
    # Create Writer agent
    writer = AssistantAgent(
        name="writer",
        model_client=client,
        tools=[
            FunctionTool(func=generate_outline, description="Create outlines"),
            FunctionTool(func=save_draft, description="Save drafts")
        ],
        system_message="""You are a professional content writer.

Create engaging, well-structured content.""",
        model_client_stream=True
    )
    
    # Create Editor agent
    editor = AssistantAgent(
        name="editor",
        model_client=client,
        tools=[
            FunctionTool(func=check_grammar, description="Check grammar"),
            FunctionTool(func=check_readability, description="Analyze readability"),
            FunctionTool(func=suggest_improvements, description="Suggest improvements")
        ],
        system_message="""You are a content editor.

Review content for quality, grammar, and readability.""",
        model_client_stream=True
    )
    
    # Create SEO agent
    seo_agent = AssistantAgent(
        name="seo_specialist",
        model_client=client,
        tools=[
            FunctionTool(func=analyze_seo, description="Analyze SEO"),
            FunctionTool(func=generate_meta_tags, description="Generate meta tags"),
            FunctionTool(func=suggest_keywords, description="Suggest keywords")
        ],
        system_message="""You are an SEO specialist.

Optimize content for search engines.""",
        model_client_stream=True
    )
    
    # Wrap agents as tools
    researcher_tool = AgentTool(agent=researcher)
    writer_tool = AgentTool(agent=writer)
    editor_tool = AgentTool(agent=editor)
    seo_tool = AgentTool(agent=seo_agent)
    
    # Create Manager with parallel_tool_calls=False (required for AgentTool)
    manager_client = OpenAIChatCompletionClient(
        model="gpt-4o-mini",
        parallel_tool_calls=False  # CRITICAL for AgentTool
    )
    
    manager = AssistantAgent(
        name="content_manager",
        model_client=manager_client,
        tools=[researcher_tool, writer_tool, editor_tool, seo_tool],
        system_message="""You are the Content Marketing Manager.

Your team:
- researcher: Gathers data and sources
- writer: Creates content
- editor: Reviews quality
- seo_specialist: Optimizes for SEO

Workflow:
1. Delegate research to researcher
2. Have writer create content based on research
3. Have editor review
4. Have SEO specialist optimize
5. Summarize final output

Coordinate efficiently!""",
        model_client_stream=True
    )
    
    # Execute workflow
    task = "Create a blog post about 'remote work trends' - 800 words"
    
    print(f"\n📋 Task: {task}\n")
    print("🚀 Manager coordinating team...\n")
    print("="*70)
    
    result = await manager.run(task=task)
    
    # Display result
    print("\n✅ WORKFLOW COMPLETED")
    print("="*70)
    
    # Show messages
    print("\n📜 Conversation Flow:")
    for i, msg in enumerate(result.messages[-5:], 1):  # Show last 5 messages
        if hasattr(msg, 'source') and hasattr(msg, 'content'):
            print(f"\n[{i}] {msg.source}:")
            content = msg.content[:200] if msg.content else ""
            print(f"    {content}...")
    
    print(f"\n\n📊 Drafts created: {len(CONTENT_DB)}")
    if CONTENT_DB:
        for draft_id, draft in CONTENT_DB.items():
            print(f"\n  {draft_id}:")
            print(f"    Title: {draft['title']}")
            print(f"    Words: {draft['word_count']}")
    
    print("\n💡 AutoGen Benefits:")
    print("   ✓ Simple AgentTool wrapper")
    print("   ✓ Automatic agent coordination")
    print("   ✓ Built-in conversation flow")
    print("\n⚠️  Note: Requires parallel_tool_calls=False")


print("✅ AutoGen content agency defined (CORRECTED)!")
print("🚀 Run with: await run_autogen_content_agency()")


✅ AutoGen content agency defined (CORRECTED)!
🚀 Run with: await run_autogen_content_agency()


In [7]:
# Simplified demo showing just Manager → Writer delegation

async def simplified_delegation_demo():
    """
    Simplified demo: Manager delegates to Writer only.
    Shows the core Agent-as-Tool pattern clearly.
    """
    from agent_framework import ai_function
    from agent_framework.openai import OpenAIChatClient
    
    print("\n" + "="*70)
    print("📝 SIMPLIFIED DELEGATION DEMO")
    print("="*70)
    print("\nPattern: Manager → Writer")
    print("-"*70)
    
    # Decorate writer tools
    @ai_function
    def generate_outline_ai(topic: str, target_length: int = 1000) -> str:
        return generate_outline(topic, target_length)
    
    @ai_function
    def save_draft_ai(title: str, content: str, author: str = "Writer") -> str:
        return save_draft(title, content, author)
    
    # Create client
    client = OpenAIChatClient(model_id="gpt-4o-mini")
    
    # Create Writer agent
    writer = client.create_agent(
        name="writer",
        instructions="""You are a creative content writer.

When asked to write:
1. Generate an outline using generate_outline_ai
2. Create engaging content following the outline
3. Save the draft using save_draft_ai

Be creative and engaging!""",
        tools=[generate_outline_ai, save_draft_ai],
        model="gpt-4.1"
    )
    
    # Convert Writer to tool
    writer_tool = writer.as_tool(
        name="creative_writer",
        description="Professional writer who creates content",
        arg_name="writing_task",
        arg_description="What to write about"
    )
    
    # Create Manager
    manager = client.create_agent(
        name="manager",
        instructions="""You are a content manager.

When given a content task:
1. Delegate it to the creative_writer
2. Present the writer's output to the user

Keep it simple and efficient!""",
        tools=[writer_tool],
        model="gpt-4o-mini"
    )
    
    # Execute
    task = "Write a short article about coffee culture (300 words)"
    
    print(f"\n👔 Manager receives task: {task}")
    print("\n💬 Manager → Writer delegation:\n")
    print("-"*70)
    
    response = ""
    async for chunk in manager.run_stream(task):
        if chunk.text:
            print(chunk.text, end="", flush=True)
            response += chunk.text
    
    print("\n\n" + "="*70)
    print("✅ SIMPLE DELEGATION COMPLETED")
    print("="*70)
    print("\n📊 What happened:")
    print("   1. Manager received task")
    print("   2. Manager delegated to Writer (agent-as-tool)")
    print("   3. Writer generated outline")
    print("   4. Writer created content")
    print("   5. Writer saved draft")
    print("   6. Manager presented result")
    
    print(f"\n💾 Drafts: {len(CONTENT_DB)}")


print("✅ Simplified demo defined!")
print("🚀 Run with: await simplified_delegation_demo()")


✅ Simplified demo defined!
🚀 Run with: await simplified_delegation_demo()


In [8]:
# Display architecture and comparison

def show_architecture():
    """Display hierarchical architecture comparison."""
    
    info = """
╔═══════════════════════════════════════════════════════════════════════════╗
║        AGENT-AS-TOOL PATTERN - CONTENT AGENCY ARCHITECTURE                ║
╚═══════════════════════════════════════════════════════════════════════════╝

🏢 ORGANIZATIONAL STRUCTURE
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

                    ┌─────────────────────┐
                    │  Content Manager    │
                    │   (Coordinator)     │
                    └──────────┬──────────┘
                               │
            ┌──────────────────┼──────────────────┐
            │                  │                  │
    ┌───────▼───────┐  ┌──────▼──────┐  ┌────────▼────────┐
    │  Researcher   │  │   Writer    │  │     Editor      │
    │  (Specialist) │  │ (Specialist)│  │  (Specialist)   │
    └───────┬───────┘  └──────┬──────┘  └────────┬────────┘
            │                  │                  │
    ┌───────▼───────┐  ┌──────▼──────┐  ┌────────▼────────┐
    │   Research    │  │   Writing   │  │    Editing      │
    │    Tools      │  │    Tools    │  │     Tools       │
    └───────────────┘  └─────────────┘  └─────────────────┘


🔄 WORKFLOW EXECUTION
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Step 1: User Request
   "Create a blog post about AI in business"
        ↓
Step 2: Manager Analyzes
   Manager thinks: "I need research, writing, editing, and SEO"
        ↓
Step 3: Delegate to Researcher
   Manager → research_specialist("Research AI in business")
   Researcher uses: research_topic(), find_citations()
   Returns: Statistics, trends, sources
        ↓
Step 4: Delegate to Writer
   Manager → content_writer("Write based on research")
   Writer uses: generate_outline(), save_draft()
   Returns: Structured article draft
        ↓
Step 5: Delegate to Editor
   Manager → content_editor("Review the draft")
   Editor uses: check_grammar(), check_readability()
   Returns: Quality report, suggestions
        ↓
Step 6: Delegate to SEO
   Manager → seo_optimizer("Optimize content")
   SEO uses: analyze_seo(), generate_meta_tags()
   Returns: SEO-optimized version
        ↓
Step 7: Manager Synthesizes
   Manager compiles all outputs
   Presents final, polished article to user


📊 FRAMEWORK COMPARISON
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Feature                 | Agent Framework    | AutoGen
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Agent Wrapping          | agent.as_tool()    | AgentTool(agent)
Customization           | ✅ Full control    | ⚠️  Limited
Tool Naming             | ✅ Custom names    | ⚠️  Auto from agent
Parallel Calls Setting  | ✅ Automatic       | ⚠️  Must disable manually
Streaming Visibility    | ✅ Detailed chunks | ⚠️  Via Console
Setup Complexity        | ⚠️  More config    | ✅ Simpler
Best For                | Production systems | Quick prototypes


💡 WHY THIS PATTERN IS POWERFUL
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

1. Specialization
   ✓ Each agent focuses on ONE domain
   ✓ Better quality than generalist
   ✓ Easier to optimize and improve

2. Modularity
   ✓ Swap out specialists easily
   ✓ Add new specialists without changing manager
   ✓ Test specialists independently

3. Reusability
   ✓ Same writer used in multiple workflows
   ✓ Researcher agent shared across projects
   ✓ Build library of specialist agents

4. Scalability
   ✓ Distribute specialists across servers
   ✓ Load balance by specialist type
   ✓ Scale expensive operations independently

5. Maintainability
   ✓ Update one specialist without touching others
   ✓ Clear separation of concerns
   ✓ Easier debugging and testing


🎯 REAL-WORLD APPLICATIONS
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Content Marketing (Our Demo):
Manager → Researcher → Writer → Editor → SEO

Software Development:
Project Manager → Architect → Developer → Tester → Reviewer

Customer Support:
Triage Agent → Technical → Billing → Escalation

Research & Analysis:
Coordinator → Literature Review → Data Analysis → Visualization → Writer

Legal Document Review:
Lead Attorney → Research → Drafting → Compliance → Finalization

Medical Diagnosis:
Primary Care → Radiologist → Pathologist → Specialist → Treatment Plan


💰 COST CONSIDERATIONS
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Single Agent Approach:
├─ 1 LLM call
├─ Cost: Low
└─ Quality: Variable (generalist)

Multi-Agent Hierarchical:
├─ 5-6 LLM calls (manager + specialists)
├─ Cost: 5-6x higher
└─ Quality: Higher (specialists)

Optimization Strategies:
├─ Use cheaper models for simple tasks
├─ Cache repeated specialist calls
├─ Parallel execution where possible
└─ Monitor and optimize per specialist


🚀 IMPLEMENTATION TIPS
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

1. Start Simple
   Begin with manager + 1 specialist
   Add specialists incrementally

2. Clear Instructions
   Each agent needs specific role definition
   Manager needs clear delegation logic

3. Error Handling
   Specialists should gracefully handle failures
   Manager should have fallback strategies

4. Monitoring
   Track which specialists are called
   Measure response times per specialist
   Monitor cost per specialist

5. Testing
   Test each specialist independently
   Test manager's delegation logic
   Test full workflow end-to-end


✨ KEY TAKEAWAYS
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

✓ Agent-as-Tool = Hierarchical AI organizations
✓ Mirrors real-world team structures
✓ Better quality through specialization
✓ More expensive but worth it for complex tasks
✓ Essential pattern for enterprise AI systems
✓ Both frameworks support it well
✓ Choose based on need for control vs simplicity
"""
    
    print(info)

show_architecture()



╔═══════════════════════════════════════════════════════════════════════════╗
║        AGENT-AS-TOOL PATTERN - CONTENT AGENCY ARCHITECTURE                ║
╚═══════════════════════════════════════════════════════════════════════════╝

🏢 ORGANIZATIONAL STRUCTURE
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

                    ┌─────────────────────┐
                    │  Content Manager    │
                    │   (Coordinator)     │
                    └──────────┬──────────┘
                               │
            ┌──────────────────┼──────────────────┐
            │                  │                  │
    ┌───────▼───────┐  ┌──────▼──────┐  ┌────────▼────────┐
    │  Researcher   │  │   Writer    │  │     Editor      │
    │  (Specialist) │  │ (Specialist)│  │  (Specialist)   │
    └───────┬───────┘  └──────┬──────┘  └────────┬────────┘
            │                  │                  │
    ┌───────▼───────┐  ┌──────▼──────┐  ┌────────▼─────

In [9]:
# Run Agent Framework Content Agency
await run_agent_framework_content_agency()


⚡ AGENT FRAMEWORK - CONTENT MARKETING AGENCY

Hierarchical Structure:
  Manager Agent
    ├── Researcher Agent
    ├── Writer Agent
    ├── Editor Agent
    └── SEO Specialist Agent

----------------------------------------------------------------------

📋 Task: Create a blog post about 'artificial intelligence in business' - 800 words

🚀 Manager delegating to specialists...

🔬 Researching topic: impact and applications of artificial intelligence in business (depth: standard)
📚 Finding citations for: impact of artificial intelligence in business
📝 Generating outline for: Artificial Intelligence in Business
📝 Generating outline for: Artificial Intelligence in Business
💾 Saving draft: The Complete Guide to Artificial Intelligence In Business
💾 Saving draft: The Complete Guide to Artificial Intelligence In Business
💾 Saving draft: The Complete Guide to Artificial Intelligence In Business
✏️  Checking grammar (1373 characters)...
📊 Analyzing readability...
💡 Generating improvement suggest

In [13]:
# Run AutoGen Content Agency
await run_autogen_content_agency()


🤖 AUTOGEN - CONTENT MARKETING AGENCY

Hierarchical Structure:
  Manager Agent
    ├── Researcher Agent
    ├── Writer Agent
    ├── Editor Agent
    └── SEO Agent

----------------------------------------------------------------------

📋 Task: Create a blog post about 'remote work trends' - 800 words

🚀 Manager coordinating team...

🔬 Researching topic: remote work trends 2023 (depth: standard)

✅ WORKFLOW COMPLETED

📜 Conversation Flow:

[1] researcher:
    [FunctionCall(id='call_2jbOBzTJB2VwyQjtEngBpJY6', arguments='{"topic":"remote work trends 2023","depth":"standard"}', name='research_topic')]...

[2] researcher:
    [FunctionExecutionResult(content='{\n  "topic": "remote work trends 2023",\n  "depth": "standard",\n  "statistics": {\n    "remote_workers_2024": "35% of US workforce",\n    "productivity_increase": "13%",\n    "cost_savings": "$11,000 per employee"\n  },\n  "current_trends": [\n    "Hybrid work models",\n    "Digital nomad lifestyle",\n    "Work-life balance focus",\

In [14]:
# Run Simplified Delegation Demo
await simplified_delegation_demo()



📝 SIMPLIFIED DELEGATION DEMO

Pattern: Manager → Writer
----------------------------------------------------------------------

👔 Manager receives task: Write a short article about coffee culture (300 words)

💬 Manager → Writer delegation:

----------------------------------------------------------------------
📝 Generating outline for: coffee culture
💾 Saving draft: The Complete Guide to Coffee Culture
### The Complete Guide to Coffee Culture

**Introduction**  
Coffee—more than just a drink, it represents a significant aspect of modern culture. The vibrant rituals surrounding coffee have transformed it into a social and communal experience. This article delves into the nuances of coffee culture, its benefits, and what trends define this beloved beverage today.  

**What is Coffee Culture**  
Coffee culture embodies the rituals, social interactions, and economic traditions surrounding coffee. From bustling coffee shops in urban centers to intimate home brewing sessions, it is the conn

🎯 What I've Built
A complete Content Marketing Agency demonstrating:

- ✅ Hierarchical Agent Architecture - Manager coordinating specialists
- ✅ 4 Specialist Agents - Researcher, Writer, Editor, SEO
- ✅ 11 Business Tools - Real functionality for each specialist
- ✅ Agent-as-Tool Pattern - Agents delegating to other agents
- ✅ Both Frameworks - AutoGen vs Agent Framework comparison
- ✅ Production Workflow - Complete content creation pipeline

see Agent Framework in action! 🚀

This showcases why hierarchical agents are powerful - instead of one agent trying to do everything, specialists collaborate like a real team, producing higher quality results through delegation and expertise!